# Data-Drivien Decision Making Using SQL

- ## Analyze Moive Data using SQL

<img width="844" alt="movie_rental" src="https://user-images.githubusercontent.com/50973416/59895563-ffeb0480-941f-11e9-9709-6a8f1a45251c.png">

## Objectives of data driven decision making
### Information for operational decisions
- Popularity of actors to decide which movies to invest in.
- Revenue ofthe last months to estimate budget for short term investments.

### Information for strategic decisions
- Success across countries to decide on market extensions.
- Longterm development of revenue for long term investments

## KPI

- Total number of rentals: revenue
- The average rating of all movies: customer satisfaction
- Number of active customers: customer engagement

## Derive KPI using OLAP



### categories of movies
list the number of movies for different genres and release years.

In [ ]:
sql = '''

SELECT 
    year_of_release,
    genre,
    COUNT(*)
FROM movies
GROUP BY CUBE (year_of_release, genre)
ORDER BY year_of_release;

'''

### Analyzing average ratings

In [1]:
sql = '''

SELECT 
    c.country, 
    m.genre, 
    AVG(r.rating) AS avg_rating 
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY CUBE (c.country, m.genre);

'''

### Number of customers

In [2]:
sql = '''

SELECT 
    country,
    gender,
    COUNT(*)
FROM customers
GROUP BY ROLLUP (country, gender)
ORDER BY country, gender;

'''

### Exploring rating by country and gender

In [3]:
sql = '''

SELECT 
    c.country, 
    c.gender,
    AVG(r.rating)
FROM renting AS r
LEFT JOIN customers AS c
ON r.customer_id = c.customer_id
GROUP BY GROUPING SETS ((country, gender), (country), (gender), ());

'''

### Customer preference for genre

In [4]:
sql ='''

SELECT 
    m.genre,
    AVG(r.rating) AS avg_rating,
    COUNT(r.rating) AS n_rating,
    COUNT(*) AS n_rentals,     
    COUNT(DISTINCT m.movie_id) AS n_movies 
FROM renting AS r
LEFT JOIN movies AS m
ON m.movie_id = r.movie_id
WHERE r.movie_id IN ( 
    SELECT 
        r.movie_id
    FROM renting
    GROUP BY r.movie_id
    HAVING COUNT(rating) >= 3)
AND r.date_renting >= '2018-01-01'
GROUP BY m.genre
ORDER BY avg_rating DESC; -- Order the table by decreasing average rating

'''

### Customer preference for actor

In [ ]:
sql = '''

SELECT 
    a.nationality,
    a.gender,
    AVG(r.rating) AS avg_rating,
    COUNT(r.rating) AS n_rating,
    COUNT(*) AS n_rentals,
    COUNT(DISTINCT a.actor_id) AS n_actors
FROM renting AS r
LEFT JOIN actsin AS ai
ON ai.movie_id = r.movie_id
LEFT JOIN actors AS a
ON ai.actor_id = a.actor_id
WHERE r.movie_id IN ( 
    SELECT 
        movie_id
    FROM renting
    GROUP BY movie_id
    HAVING COUNT(rating) >= 4)
AND r.date_renting >= '2018-04-01'
GROUP BY CUBE (a.nationality, a.gender);

'''